In [1]:
import torch
import imageio
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import os
import time
import sklearn.mixture as mixture

In [2]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]

x_max = 100
y_max = 100
z_max = 100
tfinal = 4.
source_location = np.array([[0,0,0],[.25,.75,.5]])*100

ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [3]:
sigma=.025
model = PINN([30,30,30,30,30])
model.set_location(source_location,[tfinal,x_max,y_max,z_max],source_values=[0,1],sigma=sigma)


d:\andyh\Documents\Projects\mines\methane_project\methane_pinn_dev\Net.py:12: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [4]:
x_max

100

In [5]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=1e-3)
# from torch.optim.lr_scheduler import ExponentialLR

# scheduler = ExponentialLR(optimizer, gamma=0.999)  # Decay LR by 5% every epoch

In [6]:
n= int(5e2)
# n=1
sn = int(5e3)
best_loss = np.inf
max_epochs = int(5e3)
print_freq = 100

sampling_freq = 10 # how often to resample collocation and source points

for epoch in range(max_epochs):

    start_time = time.time()

    if epoch % sampling_freq == 0:
        # points for source PDE loss. uvpoints are wind field at different times. source_values is the value of source at source_colloc_points (assumes Gaussian for now)
        source_collocation_points = model.source_points(sn,sigma*2) 
        # source_uv_points = torch.ones(len(source_collocation_points),2)*.5
        # initial condition collocation points with smaller time values 0.1*t_final. 
        # ic_col = torch.cat([torch.rand(sn,1)*tfinal*.1, torch.rand(sn,1)*x_max, torch.rand(sn,1)*y_max, torch.rand(sn,1)*z_max], dim=1)
        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max, torch.rand(n,1)*y_max, torch.rand(n,1)*z_max], dim=1)
        # collocation_points = torch.cat([collocation_points,ic_col,source_collocation_points])
        collocation_points = torch.cat([collocation_points,source_collocation_points])
        collocation_points.requires_grad_(True)
        # collocation_points.requires_grad=True
        uv = torch.ones(len(collocation_points),2)*.5#wind tensor
        uv[:,1:]*= -1
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.loss_function(collocation_points,uv) # PDE residual loss
    # loss_2,pde_2 = model.loss_function(source_collocation_points, source_uv_points) # source term PDE residual loss 
    # loss_3 ,pde_3 = model.loss_function(torch.concat([collocation_points,source_collocation_points]),torch.concat([collocation_points,source_collocation_points]))

    # loss = loss_1 + loss_2
    # loss = loss_2*100
    loss = loss_1

    # # print loss at first epoch
    # if epoch == 0:
    #     print('epoch: %d, loss: %1.3e, pde_res: %1.3e, source_loss: %1.3e' % (epoch, loss.item(), loss_1.item(), loss_2.item()))

    loss.backward()

    # compute norm of gradient of the network
    grad_norm = 0
    # for p in model.net.parameters():
    #     grad_norm += p.grad.data.norm(2).item()**2
    # grad_norm = grad_norm**0.5


    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()

    end_time = time.time()
    epoch_time = end_time - start_time
    # scheduler.step()

    if epoch % print_freq == 0:

        # print epoch and loss using %1.3e format
        # print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, source_loss: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), loss_2.item(), epoch_time))
        print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), epoch_time))

        # print(epoch, loss.item())
        # print(loss_2.item(),loss_1.item())


epoch: 0, loss: 4.937e+04, grad_norm: 0.000e+00, pde_res: 4.937e+04, time: 3.314e-01
epoch: 100, loss: 1.615e+01, grad_norm: 0.000e+00, pde_res: 1.615e+01, time: 7.368e-03
epoch: 200, loss: 1.419e+01, grad_norm: 0.000e+00, pde_res: 1.419e+01, time: 7.433e-03
epoch: 300, loss: 1.200e+01, grad_norm: 0.000e+00, pde_res: 1.200e+01, time: 8.262e-03
epoch: 400, loss: 1.039e+01, grad_norm: 0.000e+00, pde_res: 1.039e+01, time: 6.620e-03
epoch: 500, loss: 9.355e+00, grad_norm: 0.000e+00, pde_res: 9.355e+00, time: 7.386e-03
epoch: 600, loss: 9.193e+00, grad_norm: 0.000e+00, pde_res: 9.193e+00, time: 7.301e-03
epoch: 700, loss: 8.095e+00, grad_norm: 0.000e+00, pde_res: 8.095e+00, time: 8.311e-03
epoch: 800, loss: 8.237e+00, grad_norm: 0.000e+00, pde_res: 8.237e+00, time: 7.376e-03
epoch: 900, loss: 7.180e+00, grad_norm: 0.000e+00, pde_res: 7.180e+00, time: 7.179e-03
epoch: 1000, loss: 6.930e+00, grad_norm: 0.000e+00, pde_res: 6.930e+00, time: 7.557e-03
epoch: 1100, loss: 6.051e+00, grad_norm: 0.0

In [7]:
print(x_max)

100


In [8]:
model = torch.load('best_mod.m')

In [9]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = .5*(100)
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 100
x_grid = np.linspace(0, x_max, n)
y_grid = np.linspace(0, y_max, n)
z_grid = np.linspace(0, z_max, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 20)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

'\n# Plot the concentration over time\nfor t in time_steps:\n    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input\n    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape\n    \n    plt.figure()\n    plt.contourf(X, Y, concentration, levels=50, cmap=\'viridis\', vmin=0, vmax=10)  # Plot concentration as a contour plot\n    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), \'ro\', label=\'Source Location\')  # Plot the source location\n    plt.colorbar(label=\'Concentration\')\n    # fix colorbar from 0 to 1 \n    # plt.clim(0, 10.0)  # Set colorbar limits\n    plt.title(f"Gas Concentration at t = {t:.2f}")\n    plt.xlabel(\'x\')\n    plt.ylabel(\'y\')\n    # fix colorbar \n    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image\n'

In [10]:
# save as a GIF
import imageio
import os

images = []
# source_loc = torch.tensor([[.5,.5,.5]])
source_loc = model.source_locs
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=True).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[0,0], model.source_locs[0,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual_basic.gif', images)  # Save the images as a GIF



C:\Users\andyh\AppData\Local\Temp\ipykernel_17740\2046882687.py:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list


In [11]:
print(len(model.net.hidden))

6


In [12]:
print(source_points)

[[61.84826691 40.32822479  4.5       ]
 [99.10094831 54.69940709  2.        ]
 [99.89962676 24.72759871  2.        ]
 [23.54499552 57.03946784  2.        ]
 [25.09781584 22.62636785  2.        ]]
